In [ ]:
import sys
sys.path.append('../')

In [ ]:
from f1tenth_gym.f110_env import F110Env
from src.envs.wrapper import F110Wrapper
from f1tenth_gym.maps.map_manager import MapManager, MAP_DICT

map_name = 'Austin'
map_ext = '.png'
speed = 8.0
downsample = 1
use_dynamic_speed = True
a_lat_max = 3
smooth_sigma = 2

map_manager = MapManager(
    map_name=map_name,
    map_ext=map_ext,
    speed=speed,
    downsample=downsample,
    use_dynamic_speed=use_dynamic_speed,
    a_lat_max=a_lat_max,
    smooth_sigma=smooth_sigma
)

vehicle_param = {
    'mu': 1.0489,
    'C_Sf': 4.718,
    'C_Sr': 5.4562,
    'lf': 0.15875,
    'lr': 0.17145,
    'h': 0.074,
    'm': 3.74,
    'I': 0.04712,
    's_min': -0.4,
    's_max': 0.4,
    'sv_min': -3.2,
    'sv_max': 3.2,
    'v_switch': 7.319,
    'a_max': 9.51,
    'v_min': -5.0,
    'v_max': 10.0,
    'width': 0.31,
    'length': 0.58
}

num_beams = 1080
num_agents = 1
## 公式のベース環境
env = F110Env(map=map_manager.map_path, map_ext=map_ext, num_beams=num_beams, num_agents=num_agents, params=vehicle_param)
## 自作のラッパー
env = F110Wrapper(env, map_manager=map_manager)

In [ ]:
from src.planner.purePursuit import PurePursuitPlanner
wheelbase = 0.33
lookahead = 0.6
gain = 0.2
max_reacquire = 20.0

planner = PurePursuitPlanner(
    wheelbase=wheelbase,
    map_manager=map_manager,
    lookahead=lookahead,
    gain=gain,
    max_reacquire=max_reacquire
)

In [ ]:
import numpy as np
import lidar_graph
import matplotlib.pyplot as plt
import networkx as nx
from IPython.display import clear_output
import time
import math
import torch
from torch_geometric.data import Data

max_steps = 3000
num_agents = 1  # 複数エージェントの場合は変更

# グラフの初期化
lidar_graph.initialize(1080)

# 環境のリセット
obs, info = env.reset()
done = False

# LiDARデータの取得
lidar_data = obs['scans'][0].tolist()
edges = lidar_graph.build_graph(lidar_data)

# Cから座標リストを取得
positions = lidar_graph.get_node_positions()

# エッジインデックスの生成
edge_index = torch.tensor([[e[0], e[1]] for e in edges], dtype=torch.long).t().contiguous()

# 座標をテンソル形式に変換
x = torch.tensor(positions, dtype=torch.float)

# PyGのデータオブジェクトの作成
data = Data(x=x, edge_index=edge_index)

# 結果の表示
print(data)


In [ ]:
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import time
from torch_geometric.nn import SAGEConv

class GCNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNModel, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x


model = GCNModel(in_channels=2, hidden_channels=16, out_channels=2)

# GPUが使えるなら使う
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
data = data.to(device)

In [ ]:
# 推論の時間計測 N=100
model.eval()  # 推論モードに切り替え

start_time = time.time()
for _ in range(100):
    with torch.no_grad():
        out = model(data)
end_time = time.time()
inference_time = (end_time - start_time)
print(f"Inference time per sample: {inference_time:.6f} seconds")
 

In [ ]:

for step in range(max_steps):
    actions = []
    
    # エージェントごとのアクション決定
    for agent_id in range(num_agents):
        steer, speed = planner.plan(obs, id=agent_id)
        action = [steer, speed]
        actions.append(action)
    
    # 環境を一歩進める
    next_obs, reward, terminated, truncated, info = env.step(np.array(actions))
    
    # LiDARデータの更新
    lidar_data = next_obs['scans'][0].tolist()
    
    # グラフの再構築
    start = time.time()
    edges = lidar_graph.build_graph(lidar_data)
    end = time.time()
    print(f"Graph build time: {end - start:.4f} seconds")
    
    # 座標計算 (-135度から135度を分割)
    positions = {}
    for i, dist in enumerate(lidar_data):
        angle = (-135.0 + i * angle_increment) * (math.pi / 180)
        x = dist * math.cos(angle)
        y = dist * math.sin(angle)
        positions[i] = (x, y)

    # NetworkXで可視化
    G = nx.Graph()
    G.add_weighted_edges_from(edges)
    clear_output(wait=True)
    plt.figure(figsize=(8, 8))
    nx.draw(G, pos=positions, with_labels=False, node_size=10, font_size=8)
    plt.pause(0.001)  # 少し待つことで描画を見やすくする
    plt.show()

    env.render()
    
    # エピソード終了判定
    if terminated or truncated:
        print(f"Episode finished after {step + 1} timesteps")
        break

    obs = next_obs